In [1]:
from ipywidgets import interact
import ipywidgets as widgets
import imgaug.augmenters as iaa
from torchvision.transforms import Compose
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import cv2

import sys
sys.path.append('../../')

from src.features.pixelwise_loss import PixelwiseLoss

from src import MODELS_DIR, MLFLOW_TRACKING_URI, DATA_PATH
from src.data import TrainValTestSplitter, MURASubset
from src.data.transforms import GrayScale, Resize, HistEqualisation, MinMaxNormalization, ToTensor
from src.features.augmentation import Augmentation
from src.models.autoencoders import BottleneckAutoencoder, BaselineAutoencoder, SkipConnection
from src.models.autoencoders import MaskedMSELoss

%matplotlib inline

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [43]:
run_params = {
    'image_resolution': (512, 512),
    'pipeline': {
        'hist_equalisation': False,
        'data_source': 'XR_HAND_PHOTOSHOP',
    }
}

augmentation_seq = iaa.Sequential([iaa.Affine(fit_output=False, rotate=(0), order=[0, 1]),  
                                   iaa.PadToFixedSize(*run_params['image_resolution'], position='center')])

composed_transforms = Compose([GrayScale(),
                               HistEqualisation(active=run_params['pipeline']['hist_equalisation']),
                               Resize(run_params['image_resolution'], keep_aspect_ratio=True),
                               Augmentation(augmentation_seq),
                               MinMaxNormalization(),
                               ToTensor()])

In [44]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.load("/home/ubuntu/mlruns/1/5ca7f67c33674926a00590752c877fe5/artifacts/BaselineAutoencoder.pth", map_location=device)
# model = torch.load("/home/ubuntu/mlruns/2/3766754ed32043bcbdc61d053615af6d/artifacts//BottleneckAutoencoder.pth", map_location=device)
# model = torch.load("/home/diana/xray/mlruns/1/28737a0f69dd4016a055433176df6a87/artifacts/VAE/data/model.pth", map_location=device)

# set loss function
outer_loss = nn.MSELoss(reduction='none')
model.eval().to(device)

BaselineAutoencoder(
  (encoder): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(16, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (13): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14

In [49]:
# Loading image
image_path = f'{DATA_PATH}/{run_params["pipeline"]["data_source"]}/patient00141/study1_negative/image2_cropped_1.png'

validation = MURASubset(filenames=[image_path], true_labels=[0],
                        patients=['00141'], transform=composed_transforms)
val_loader = DataLoader(validation, batch_size=1, shuffle=True, num_workers=2)

label = 0
patient = '00141'

evaluation = PixelwiseLoss(model=model, model_class='CAE', device=device, loss_function=MaskedMSELoss(reduction="none"), masked_loss_on_val=True)
loss_dict = evaluation.get_loss(data=val_loader)

for batch in validation:
    inp_image = batch['image'].cpu()[0]
loss = loss_dict['loss'][0][0]
loss = cv2.resize(loss, (512, 512))
inp_image = cv2.resize(inp_image.numpy(), (512, 512))


evaluation: 100%|██████████| 1/1 [00:00<00:00,  4.73it/s]


In [50]:
def plot_pixelwise(max_loss):
    evaluation.add_heatmap(inp_image, label, patient, loss, original_path=None, max_loss=max_loss, 
                           sigma=4, path=None, save=False, display=True, figsize=(5, 5))
interact(plot_pixelwise, 
         max_loss = widgets.FloatSlider(min=0.0001, max=0.005, step=0.0001, value = 0.002, readout_format ='.5f'))

interactive(children=(FloatSlider(value=0.002, description='max_loss', max=0.005, min=0.0001, readout_format='…

<function __main__.plot_pixelwise(max_loss)>

In [51]:
# Loading image
image_path = f'{DATA_PATH}/{run_params["pipeline"]["data_source"]}/patient09769/study1_positive/image2_undetected_1.png'

validation = MURASubset(filenames=[image_path], true_labels=[1],
                        patients=['09769'], transform=composed_transforms)
val_loader = DataLoader(validation, batch_size=1, shuffle=True, num_workers=2)

label = 1
patient = '09769'

evaluation = PixelwiseLoss(model=model, model_class='CAE', device=device, loss_function=MaskedMSELoss(reduction="none"), masked_loss_on_val=True)
loss_dict = evaluation.get_loss(data=val_loader)

for batch in validation:
    inp_image = batch['image'].cpu()[0]
loss = loss_dict['loss'][0][0]
loss = cv2.resize(loss, (512, 512))
inp_image = cv2.resize(inp_image.numpy(), (512, 512))


evaluation: 100%|██████████| 1/1 [00:00<00:00,  1.83it/s]


In [52]:
def plot_pixelwise(max_loss):
    evaluation.add_heatmap(inp_image, label, patient, loss, original_path=None, max_loss=max_loss, 
                           sigma=4, path=None, save=False, display=True, figsize=(5, 5))
interact(plot_pixelwise, 
         max_loss = widgets.FloatSlider(min=0.0001, max=0.01, step=0.0005, value = 0.001, readout_format ='.5f'))

interactive(children=(FloatSlider(value=0.001, description='max_loss', max=0.01, min=0.0001, readout_format='.…

<function __main__.plot_pixelwise(max_loss)>